# Data Augmentation

Here’s an easy and very cool trick to increase the size of our dataset called data augmentation. The idea is to randomly transform the input images through rotations, scaling, mirroring, etc. This helps the network generalize to images not in the training set.

# Sample Code

We have some model set-up code which you've seen before.  It's not our focus for the moment, so it is hidden (but optionally visible by clicking the "code" button below.)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting a Model With Data Augmentation

If you flip an image of an urban scene horizontally, taking the photos mirror image, it would still look urban.  So, we can train our model with both the original image and with the mirror image. We do this with with the `horizontal_flip` argument to `ImageDataGenerator`.  If we set `horizontal_flip=True`, the `ImageDataGenerator` will randomly decide whether or not to flip an image every time it is about to send the image to the model training.

Before we had 72 images, now it’s almost as if we had twice as many.

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224

data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

train_generator = data_generator_with_aug.flow_from_directory(
        '../input/urban-and-rural-photos/train',
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

There are some cases where the mirror image of something shouldn’t be classified the same as the original image.  The mirror image of a stop sign doesn’t say stop, so maybe we shouldn’t train a model to think that’s a stop sign. So you need to judge whether this makes sense on a case by case basis.

There are other similar data augmentation techniques listed in the `ImageDataGenerator`, which you can see in Kernels by typing `ImageDataGenerator` and then a question mark.  For instance we could slightly crop the photo, effectively shifting it slightly horizontally, or slightly vertically.  The keyword arguments `width_shift_range` and `height_shift_range` randomly shift the part of the image further to either side and further vertically, respectively. Here I've set both arguments to 0.2 which shifts the images by 20% in either direction.

Even though I have 72 raw images, we’ll get many different variations of them when model training. At some point, we may have so many raw images that we don’t need this, but it’s usually valuable even with hundreds of thousands of images.

### Validation images

To do an apples-to-apples comparison against the previous model, where we didn’t do data augmentation, I’ll use an `ImageDataGenerator` for our validation data that doesn’t do this type of data augmentation or manipulation.  For measuring model quality, we’ll use the same raw images as we used before.

In [ ]:
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator = data_generator_no_aug.flow_from_directory(
        '../input/urban-and-rural-photos/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

Now we’re ready to fit the model. 

In [ ]:
my_new_model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        epochs=2,
        validation_data=validation_generator,
        validation_steps=1)

Since we get different versions of the images each time we see them, I’ll add the argument epochs equals two to my call to fit the model.  That means it goes through each raw image file two times. You can experiment with different numbers of epochs. Just remember that data augmentation allows us to use more epochs before we start overfitting, and seeing validation scores get worse.

In this particular case, our validation data is so small that there’s a little bit of noise or luck in the score from any given model.  Validation scores are more reliable on bigger validation datasets, and that’s something we’ll get to.

# Exercise
Move on to **[apply data augmentation](#$EXERCISE_FORKING_URL$)** yourself.